In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import thinkstats2
import thinkplot

import statsmodels.formula.api as smf

import pickle

In [2]:
dc_df = pd.read_csv('dc_comic.csv')
marvel_df = pd.read_csv('marvel_comic.csv')
# both_df = pd.read_csv('both_comic.csv')

In [16]:
dc_df['dcmar'] = 1
marvel_df['dcmar'] = 0
marvel_df['YEAR'] = marvel_df['Year']
marvel_df = marvel_df.drop('Year', axis=1)

frames = [dc_df, marvel_df]
both_df = pd.concat(frames, sort=False)

In [17]:
dc_df.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,YEAR,dcmar
0,1422,597,597,2.0,1.0,3.0,0.0,2.0,NaN,1.0,3093.0,1939.0,1
1,23387,6007,6006,2.0,1.0,3.0,0.0,2.0,NaN,1.0,2496.0,1986.0,1
2,1458,2487,2487,2.0,1.0,4.0,3.0,2.0,NaN,1.0,1565.0,1959.0,1
3,1659,3002,3003,1.0,1.0,4.0,16.0,2.0,NaN,1.0,1316.0,1987.0,1
4,1576,5280,5279,2.0,1.0,3.0,0.0,2.0,NaN,1.0,1237.0,1940.0,1


In [18]:
dc_df.describe()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,YEAR,dcmar
count,6896.000000,6896.000000,6896.000000,4883.000000,6295.000000,3268.000000,4622.000000,6771.000000,64.000000,6893.000000,6541.000000,6827.000000,6896.0
mean,147441.209252,3447.500000,3447.500000,1.491296,0.630818,5.110159,3.710299,1.416187,0.843750,0.754389,23.625134,1989.766662,1.0
std,108388.631149,1990.848061,1990.848061,0.503649,0.644453,3.849095,4.904919,0.907965,0.365963,0.430480,87.378509,16.824194,0.0
min,1380.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1935.000000,1.0
25%,44105.500000,1723.750000,1723.750000,1.000000,0.000000,3.000000,0.000000,0.000000,1.000000,1.000000,2.000000,1983.000000,1.0
50%,141267.000000,3447.500000,3447.500000,1.000000,1.000000,4.000000,1.000000,2.000000,1.000000,1.000000,6.000000,1992.000000,1.0
75%,213203.000000,5171.250000,5171.250000,2.000000,1.000000,6.000000,3.000000,2.000000,1.000000,1.000000,15.000000,2003.000000,1.0
max,404010.000000,6895.000000,6895.000000,2.000000,3.000000,16.000000,16.000000,3.000000,1.000000,1.000000,3093.000000,2013.000000,1.0


In [19]:
both_df.columns

Index(['page_id', 'name', 'urlslug', 'ID', 'ALIGN', 'EYE', 'HAIR', 'SEX',
       'GSM', 'ALIVE', 'APPEARANCES', 'YEAR', 'dcmar'],
      dtype='object')

In [20]:
marvel_df.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,dcmar,YEAR
0,1678,9765,9764,3.0,1.0,9.0,6.0,3.0,NaN,1.0,4043.0,0,1962.0
1,7139,1629,1629,2.0,1.0,3.0,22.0,3.0,NaN,1.0,3360.0,0,1941.0
2,64786,11142,11142,2.0,2.0,3.0,2.0,3.0,NaN,1.0,3061.0,0,1974.0
3,1868,4735,4735,2.0,1.0,3.0,2.0,3.0,NaN,1.0,2961.0,0,1963.0
4,2460,10256,10256,1.0,1.0,3.0,3.0,3.0,NaN,1.0,2258.0,0,1950.0


In [21]:
marvel_df.describe()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,dcmar,YEAR
count,11399.000000,11399.000000,11399.000000,9178.000000,9750.000000,5228.000000,8989.000000,10957.000000,81.000000,11398.000000,11398.000000,11399.0,11024.000000
mean,262539.338275,5699.000000,5699.000000,2.367182,0.663282,6.910865,6.555568,2.471753,1.617284,0.754782,22.494122,0.0,1983.363661
std,247756.968509,3290.752194,3290.752194,0.724318,0.727065,6.349409,6.159354,0.886623,1.019410,0.430236,111.058174,0.0,20.284986
min,1025.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,1939.000000
25%,18709.000000,2849.500000,2849.500000,2.000000,0.000000,3.000000,2.000000,1.000000,2.000000,1.000000,2.000000,0.0,1972.000000
50%,236126.000000,5699.000000,5699.000000,3.000000,1.000000,4.000000,3.000000,3.000000,2.000000,1.000000,5.000000,0.0,1988.000000
75%,485393.500000,8548.500000,8548.500000,3.000000,1.000000,7.000000,10.000000,3.000000,2.000000,1.000000,12.000000,0.0,1999.000000
max,754840.000000,11398.000000,11398.000000,3.000000,2.000000,23.000000,23.000000,3.000000,5.000000,1.000000,4043.000000,0.0,2013.000000


In [22]:
marvel_df.describe()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,dcmar,YEAR
count,11399.000000,11399.000000,11399.000000,9178.000000,9750.000000,5228.000000,8989.000000,10957.000000,81.000000,11398.000000,11398.000000,11399.0,11024.000000
mean,262539.338275,5699.000000,5699.000000,2.367182,0.663282,6.910865,6.555568,2.471753,1.617284,0.754782,22.494122,0.0,1983.363661
std,247756.968509,3290.752194,3290.752194,0.724318,0.727065,6.349409,6.159354,0.886623,1.019410,0.430236,111.058174,0.0,20.284986
min,1025.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,1939.000000
25%,18709.000000,2849.500000,2849.500000,2.000000,0.000000,3.000000,2.000000,1.000000,2.000000,1.000000,2.000000,0.0,1972.000000
50%,236126.000000,5699.000000,5699.000000,3.000000,1.000000,4.000000,3.000000,3.000000,2.000000,1.000000,5.000000,0.0,1988.000000
75%,485393.500000,8548.500000,8548.500000,3.000000,1.000000,7.000000,10.000000,3.000000,2.000000,1.000000,12.000000,0.0,1999.000000
max,754840.000000,11398.000000,11398.000000,3.000000,2.000000,23.000000,23.000000,3.000000,5.000000,1.000000,4043.000000,0.0,2013.000000


In [23]:
both_df = both_df.drop('FIRST APPEARANCE', axis=1)
marvel_df = marvel_df.drop('FIRST APPEARANCE', axis=1)
dc_df = dc_df.drop('FIRST APPEARANCE', axis=1)

KeyError: "['FIRST APPEARANCE'] not found in axis"

In [24]:
def map_cat_to_number(variable):
    """Creates a mapping from a categorical label to an integer.
    
    Args:
        variable (pandas Series): The categorical variable to map.
    
    Returns:
        cat_dict (dictionary): A dictionary with keys as the categorical
            labels and values as the integers corresponding to the label.
        mapped_variable (pandas Series): The variable with categorical 
            labels replaced with numbers.
    """
    categories = list(variable.cat.categories)
    cat_dict = {}
    for code, cat in enumerate(categories):
        cat_dict[cat] = code
    
    mapped_variable = variable.map(cat_dict)
    return cat_dict, mapped_variable

In [25]:
def object_types_to_numeric(df):
    """Converts columns of type 'object' to numeric values.
    
    Args:
        df (pandas DataFrame): The data frame in which to convert columns.
        
    Returns:
        df (pandas DataFrame): The data frame with converted columns.
    """
    object_columns = df.select_dtypes(include=['object']).columns
    cat_dicts = {}
    for col in df.columns:
        if col in object_columns:
            cat_dicts[col], df[col] = map_cat_to_number(df[col].astype('category'))
            
    return cat_dicts, df

In [26]:
# Convert data frames to saveable formats.
cat_dicts_both, both_df = object_types_to_numeric(both_df)
cat_dicts_marvel, marvel_df = object_types_to_numeric(marvel_df)
cat_dicts_dc, dc_df = object_types_to_numeric(dc_df)

In [27]:
# Save categorical mappings.
with open('cat_dicts_both.pkl', 'wb') as f:
    pickle.dump(cat_dicts_both, f)

with open('cat_dicts_marvel.pkl', 'wb') as f:
    pickle.dump(cat_dicts_marvel, f)
    
with open('cat_dicts_dc.pkl', 'wb') as f:
    pickle.dump(cat_dicts_dc, f)

In [28]:
# Save the data frames as hdf files.
both_df.to_hdf('both_df.hdf5', mode='w', key='both')
marvel_df.to_hdf('marvel_df.hdf5', mode='w', key='marvel')
dc_df.to_hdf('dc_df.hdf5', mode='w', key='dc')